In [1]:
import random
import math
import heapq
import tkinter as tk
from tkinter import ttk, messagebox
from dataclasses import dataclass
from typing import List, Optional

In [2]:
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Cell:
    def __init__(self, position, parent=None, status='free', g=0, h=0, f=0):
        self.position = position
        self.parent = parent
        self.status = status
        self.g = g
        self.h = h
        self.f = f

In [3]:
class RobotCleaner:
    def __init__(self, rows, cols, num_dirty):
        self.rows = rows
        self.cols = cols
        self.num_dirty = num_dirty

        self.move_cost_per_step = 1 
        
        self.grid = []
        self.robot_pos = None
        self.dirty_cells = []
        self.total_move_cost = 0
        self.total_remaining_dirty_cost = 0
        self.initialize_grid()

    def initialize_grid(self):
        self.grid = [[Cell(Position(x, y)) for y in range(self.cols)] for x in range(self.rows)]

        # đặt robot
        rx, ry = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
        self.robot_pos = Position(rx, ry)
        self.grid[rx][ry].status = 'start'

        # đặt ô bẩn
        count = 0
        while count < self.num_dirty:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'dirty'
                self.dirty_cells.append(Position(x, y))
                count += 1

    def heuristic(self, pos1, pos2):
        return max(abs(pos1.x - pos2.x), abs(pos1.y - pos2.y)) 
    '''Chebyshev cho lưới 8 hướng, mỗi bước = 1'''
    
    def get_neighbors(self, cell):
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),           (0, 1),
            (1, -1),  (1, 0),  (1, 1)
        ]
        neighbors = []
        for dx, dy in directions:
            nx, ny = cell.position.x + dx, cell.position.y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols:
                neighbors.append(self.grid[nx][ny])
        return neighbors

    def reset_cells(self):
        for x in range(self.rows):
            for y in range(self.cols):
                c = self.grid[x][y]
                c.g = c.h = c.f = 0.0
                c.parent = None

    def find_path(self, start, goal):
        self.reset_cells()
        start_cell = self.grid[start.x][start.y]
        goal_cell = self.grid[goal.x][goal.y]
        open_list = []
        closed = set()

        push_counter = 0

        start_cell.g = 0
        start_cell.h = self.heuristic(start, goal)
        start_cell.f = start_cell.g + start_cell.h
        heapq.heappush(open_list, (start_cell.f, push_counter, start_cell))

        while open_list:
            f, _, current = heapq.heappop(open_list)
            if (current.position.x == goal_cell.position.x and current.position.y == goal_cell.position.y): 
                '''so sánh theo tọa độ thay vì object identity'''
                path = []
                while current:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]

            closed.add((current.position.x, current.position.y))
            for neighbor in self.get_neighbors(current):
                npos = neighbor.position
                if (npos.x, npos.y) in closed:
                    continue

                temp_g = current.g + 1
                '''mỗi bước = 1, không dùng Euclid cho g'''

                #temp_g = current.g + self.heuristic(current.position, npos)
                if neighbor.parent is None or temp_g < neighbor.g:
                    neighbor.parent = current
                    neighbor.g = temp_g
                    neighbor.h = self.heuristic(npos, goal_cell.position)
                    neighbor.f = neighbor.g + neighbor.h

                    push_counter += 1
                    heapq.heappush(open_list, (neighbor.f, push_counter, neighbor))
                    
        return None

    def print_map(self, current_robot_pos=None):
        rp = current_robot_pos if current_robot_pos else self.robot_pos
        
        for x in range(self.rows):
            row = []
            for y in range(self.cols):
                if rp.x == x and rp.y == y:
                    row.append("R")
                elif self.grid[x][y].status == "dirty":
                    row.append("D")
                elif self.grid[x][y].status == "clean":
                    row.append("C")
                else:
                    row.append(".")
            print(" ".join(row))
        print()

    def clean_all(self):
        self.total_move_cost = 0
        self.total_remaining_dirty_cost = 0
        current_pos = self.robot_pos

        print("Bản đồ ban đầu:")
        self.print_map(current_pos)
        print('-----------------------------------------')

        while self.dirty_cells:
            nearest = min(self.dirty_cells, key=lambda d: self.heuristic(current_pos, d))

            path = self.find_path(current_pos, nearest)
            if not path:
                print(f"Không tìm thấy đường tới ô ({nearest.x}, {nearest.y})")
                self.dirty_cells.remove(nearest)
                continue

            move_steps = len(path) - 1
            move_cost = move_steps * self.move_cost_per_step
            self.total_move_cost += move_cost

            # tăng chi phí cho các ô bẩn còn lại theo số bước di chuyển
            remaining_dirty_cost = (len(self.dirty_cells) - 1) * move_steps
            self.total_remaining_dirty_cost += remaining_dirty_cost

            remaining_others = len(self.dirty_cells) - 1
            if remaining_others > 0:
                self.total_remaining_dirty_cost += remaining_others * move_steps # phạt trong lúc di chuyển
                self.total_remaining_dirty_cost += remaining_others * 1 # phạt thêm 1 lần do hút bụi (Suck)


            self.grid[nearest.x][nearest.y].status = "clean"
            self.dirty_cells.remove(nearest)
            current_pos = nearest
            self.robot_pos = current_pos

            print(f"Dọn ô ({nearest.x}, {nearest.y})")
            self.print_map(current_pos)
            print(f"Chi phí di chuyển: {self.total_move_cost:.2f}")
            print(f"Chi phí ô bẩn còn lại: {self.total_remaining_dirty_cost:.2f}")
            print('-----------------------------------------')

        print("Hoàn thành!")
        print(f"Tổng chi phí: {self.total_move_cost + self.total_remaining_dirty_cost:.2f}")

# if __name__ == "__main__":
#     m = int(input("Nhập số hàng: "))
#     n = int(input("Nhập số cột: "))
#     num_dirty = int(input("Nhập số lượng ô bẩn: "))

#     robot = RobotCleaner(m, n, num_dirty)
#     robot.clean_all()

In [4]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Robot Cleaner — A*")
        self.resizable(False, False)

        self.rows_var  = tk.IntVar(value=8)
        self.cols_var  = tk.IntVar(value=10)
        self.dirty_var = tk.IntVar(value=6)
        self.seed_var  = tk.IntVar(value=42)
        self.delay_var = tk.IntVar(value=250)  

        self.columnconfigure(0, weight=1)
        self.rowconfigure(2, weight=1)

        setup = ttk.LabelFrame(self, text="Setup", padding=10)
        setup.grid(row=0, column=0, sticky="ew", padx=10, pady=(10,6))
        setup.columnconfigure(9, weight=1)

        ttk.Label(setup, text="Rows").grid(row=0, column=0, padx=(0,4), sticky="w")
        ttk.Spinbox(setup, from_=3, to=60, textvariable=self.rows_var, width=5).grid(row=0, column=1, padx=(0,10))
        ttk.Label(setup, text="Cols").grid(row=0, column=2, padx=(0,4))
        ttk.Spinbox(setup, from_=3, to=60, textvariable=self.cols_var, width=5).grid(row=0, column=3, padx=(0,10))
        ttk.Label(setup, text="Dirty").grid(row=0, column=4, padx=(0,4))
        ttk.Spinbox(setup, from_=1, to=500, textvariable=self.dirty_var, width=6).grid(row=0, column=5, padx=(0,10))
        ttk.Label(setup, text="Seed").grid(row=0, column=6, padx=(0,4))
        ttk.Entry(setup, textvariable=self.seed_var, width=8).grid(row=0, column=7, padx=(0,10))
        ttk.Label(setup, text="Delay (ms)").grid(row=0, column=8, padx=(0,6))
        ttk.Scale(setup, from_=0, to=1500, variable=self.delay_var, orient="horizontal")\
            .grid(row=0, column=9, sticky="ew")

        btns = ttk.Frame(self, padding=(10,0,10,10))
        btns.grid(row=1, column=0, sticky="ew")
        self.run_btn   = ttk.Button(btns, text="Generate & Run", command=self.start_run)
        self.pause_btn = ttk.Button(btns, text="Pause", command=self.toggle_pause, state="disabled")
        self.stop_btn  = ttk.Button(btns, text="Stop", command=self.stop_run, state="disabled")
        self.run_btn.grid(row=0, column=0, padx=3)
        self.pause_btn.grid(row=0, column=1, padx=3)
        self.stop_btn.grid(row=0, column=2, padx=3)

        main = ttk.Frame(self, padding=10)
        main.grid(row=2, column=0, sticky="nsew")
        main.columnconfigure(0, weight=1)
        main.rowconfigure(0, weight=1)

        map_frame = ttk.LabelFrame(main, text="Map", padding=8)
        map_frame.grid(row=0, column=0, sticky="nsew", padx=(0,10))
        map_frame.columnconfigure(0, weight=1)
        map_frame.rowconfigure(0, weight=1)

        self.cell_size = 26
        self.canvas = tk.Canvas(map_frame, width=26*10, height=26*8, bg="#ffffff",
                                highlightthickness=1, highlightbackground="#bbb")
        self.canvas.grid(row=0, column=0, sticky="nsew")

        side = ttk.Frame(main, padding=0)
        side.grid(row=0, column=1, sticky="n")

        costs = ttk.LabelFrame(side, text="Live Costs", padding=10)
        costs.grid(row=0, column=0, sticky="ew")
        self.step_lbl  = ttk.Label(costs, text="Step: 0");       self.step_lbl.grid(row=0, column=0, sticky="w")
        self.dleft_lbl = ttk.Label(costs, text="Dirty left: 0"); self.dleft_lbl.grid(row=1, column=0, sticky="w")
        self.move_lbl  = ttk.Label(costs, text="Move: 0");       self.move_lbl.grid(row=2, column=0, sticky="w")
        self.delay_lbl = ttk.Label(costs, text="Delay: 0");      self.delay_lbl.grid(row=3, column=0, sticky="w")
        self.total_lbl = ttk.Label(costs, text="Total: 0", font=("Segoe UI", 10, "bold"))
        self.total_lbl.grid(row=4, column=0, sticky="w")

        self.show_path_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(costs, text="Show path", variable=self.show_path_var,
                        command=self.draw_path_once).grid(row=5, column=0, sticky="w", pady=(6,0))

        ttk.Separator(side, orient="horizontal").grid(row=1, column=0, sticky="ew", pady=10)

        legend = ttk.LabelFrame(side, text="Color Legend", padding=10)
        legend.grid(row=2, column=0, sticky="ew")

        color_items = [
            ("#222222", "Robot (R)"),
            ("#6ab8ff", "Start"),
            ("#ff6b6b", "Dirty"),
            ("#6bd66b", "Clean"),
            ("#eeeeee", "Free"),
        ]
        for i, (color, label) in enumerate(color_items):
            row = ttk.Frame(legend)
            row.grid(row=i, column=0, sticky="w", pady=2)
            box = tk.Canvas(row, width=18, height=18, bg=color,
                            highlightthickness=1, highlightbackground="#666")
            box.grid(row=0, column=0, padx=(0,6))
            ttk.Label(row, text=label).grid(row=0, column=1, sticky="w")

        self.rc: Optional[RobotCleaner] = None
        self.running = False
        self.paused = False
        self.after_id = None

        self.step_no = 0
        self.move_cost = 0
        self.delay_cost = 0
        self.current_path: List[Position] = []
        self.path_idx = 1

        self.colors = {
            'free':  "#eeeeee",
            'dirty': "#ff6b6b",
            'clean': "#6bd66b",
            'start': "#6ab8ff",
            'robot': "#222222",
            'path':  "#ffd166",
        }

        self.cell_rect: List[List[Optional[int]]] = []
        self.robot_id: Optional[int] = None
        self.robot_text_id: Optional[int] = None
        self.path_item_id: Optional[int] = None


    def init_grid_canvas(self):
        if not self.rc: return
        rows, cols = self.rc.rows, self.rc.cols
        s = self.cell_size
        self.canvas.config(width=cols*s, height=rows*s)
        self.canvas.delete("all")
        self.cell_rect = [[None]*cols for _ in range(rows)]

        for i in range(rows):
            for j in range(cols):
                x0, y0 = j*s, i*s
                x1, y1 = x0+s, y0+s
                st = self.rc.grid[i][j].status
                fill = self.colors.get(st, "#eee")
                rid = self.canvas.create_rectangle(x0, y0, x1, y1, fill=fill, outline="#999")
                self.cell_rect[i][j] = rid

        rx, ry = self.rc.robot_pos.x, self.rc.robot_pos.y
        x0, y0 = ry*s, rx*s
        x1, y1 = x0+s, y0+s
        self.robot_id = self.canvas.create_rectangle(x0, y0, x1, y1,
                                                     fill=self.colors['robot'], outline="#000")
        self.robot_text_id = self.canvas.create_text((x0+x1)//2, (y0+y1)//2,
                                                     text="R", fill="#fff", font=("Arial", 10, "bold"))

    def update_cell_color(self, i: int, j: int, status: str):
        rid = self.cell_rect[i][j]
        if rid is not None:
            self.canvas.itemconfig(rid, fill=self.colors.get(status, "#eee"))

    def move_robot_to(self, pos: Position):
        s = self.cell_size
        x0, y0 = pos.y*s, pos.x*s
        x1, y1 = x0+s, y0+s
        if self.robot_id is not None:
            self.canvas.coords(self.robot_id, x0, y0, x1, y1)
        if self.robot_text_id is not None:
            self.canvas.coords(self.robot_text_id, (x0+x1)//2, (y0+y1)//2)

    def _clear_path(self):
        if self.path_item_id:
            self.canvas.delete(self.path_item_id)
            self.path_item_id = None

    def draw_path_once(self):
        if not self.show_path_var.get():
            self._clear_path()
            return

        if not (self.rc and self.current_path) or self.path_idx >= len(self.current_path):
            self._clear_path()
            return

        s = self.cell_size

        rx, ry = self.rc.robot_pos.x, self.rc.robot_pos.y
        pts = [ry*s + s/2, rx*s + s/2]

        steps = self.current_path[self.path_idx:self.path_idx+5]
        for p in steps:
            pts.extend([p.y*s + s/2, p.x*s + s/2])

        if len(pts) < 4 and self.path_idx < len(self.current_path):
            g = self.current_path[-1]
            pts.extend([g.y*s + s/2, g.x*s + s/2])

        if len(pts) < 4:
            self._clear_path()
            return

        try:
            if self.path_item_id:
                self.canvas.coords(self.path_item_id, *pts)
            else:
                self.path_item_id = self.canvas.create_line(*pts, width=2)
        except tk.TclError:
            self._clear_path()


    def update_side(self):
        self.step_lbl.config(text=f"Step: {self.step_no}")
        dleft = len(self.rc.dirty_cells) if self.rc else 0
        self.dleft_lbl.config(text=f"Dirty left: {dleft}")
        self.move_lbl.config(text=f"Move: {int(self.move_cost)}")
        self.delay_lbl.config(text=f"Delay: {int(self.delay_cost)}")
        self.total_lbl.config(text=f"Total: {int(self.move_cost + self.delay_cost)}")

    def start_run(self):
        if self.running: return
        try:
            random.seed(self.seed_var.get())
            self.rc = RobotCleaner(self.rows_var.get(), self.cols_var.get(),
                                   self.dirty_var.get())
        except Exception as e:
            messagebox.showerror("Error", str(e))
            return

        self.running = True
        self.paused = False
        self.step_no = 0
        self.move_cost = 0
        self.delay_cost = 0
        self.current_path = []
        self.path_idx = 1

        self.run_btn.config(state="disabled")
        self.pause_btn.config(text="Pause", state="normal")
        self.stop_btn.config(state="normal")

        self.init_grid_canvas()
        self.update_side()
        self.schedule_tick(50)

    def toggle_pause(self):
        if not self.running: return
        self.paused = not self.paused
        self.pause_btn.config(text="Resume" if self.paused else "Pause")

    def stop_run(self):
        self.running = False
        self.paused = False
        self.run_btn.config(state="normal")
        self.pause_btn.config(text="Pause", state="disabled")
        self.stop_btn.config(state="disabled")
        if self.after_id:
            self.after_cancel(self.after_id)
            self.after_id = None

    def schedule_tick(self, ms: int):
        self.after_id = self.after(ms, self.tick)

    def tick(self):
        if not self.running or not self.rc:
            return

        if self.paused:
            self.schedule_tick(100)
            return

        if not self.rc.dirty_cells:
            self.stop_run()
            return

        if not self.current_path:
            cur = self.rc.robot_pos
            target = min(self.rc.dirty_cells, key=lambda d: self.rc.heuristic(cur, d))
            path = self.rc.find_path(cur, target)
            if not path:
                self.rc.dirty_cells.remove(target)
                self.schedule_tick(20)
                return
            self.current_path = path
            self.path_idx = 1  
            self.draw_path_once()

        if self.path_idx < len(self.current_path):
            nxt = self.current_path[self.path_idx]

            self.move_cost += self.rc.move_cost_per_step
            self.delay_cost += len(self.rc.dirty_cells)

            self.rc.robot_pos = nxt
            self.move_robot_to(self.rc.robot_pos)
            self.step_no += 1
            self.path_idx += 1

            if self.path_idx == len(self.current_path):
                target = self.current_path[-1]
                self.rc.grid[target.x][target.y].status = "clean"
                self.rc.dirty_cells = [d for d in self.rc.dirty_cells
                                       if not (d.x == target.x and d.y == target.y)]
                self.update_cell_color(target.x, target.y, "clean")
                self.delay_cost += len(self.rc.dirty_cells)
                self.current_path = []
                self.path_idx = 1
                self.draw_path_once()
            else:
                self.draw_path_once()

            self.update_side()

        d = max(0, int(self.delay_var.get()))
        self.schedule_tick(d)

In [5]:
if __name__ == "__main__":
    app = App()
    app.mainloop()